In [1]:
# ------------------------------------------
# FORMATTING OF DATAFRAME VISUALISATION
# ------------------------------------------
import pandas as pd

# A function to make pandas show an entire dataframe in a readable manner
def pretty_data(df):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df)


In [1]:
# ------------------------------------------
# WEATHER DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the weather data
weather_time = '2024-01-30/2024-03-19' # The stretch of time to fetch weather data for
weather_station = 'SN68860' # The ID of the weather station to fetch data from, can be multiple stations separated by a comma - 'SN68860' is Voll weatherstation
weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)' # The measurements to include in the dataset, here: temperautre, precipitation, and wind speed
weather_resolution = 'P1D' # The level of granularity in the weather data. P1D is once per day. P1H is once per hour.

# Run the module for importing the weather data, and save it to a dataframe
%run met.ipynb
df_weather = get_met()

# Run the module for cleaning up and checking the weather data
'''Per nå sjekker den bare om det mangler data noe sted'''
%run weather_data.ipynb
missing_weather(df_weather)

# Displaying the weather data as a table
'''Fjern # under for å se datasettet'''
#pretty_data(df_weather)

Data collected from frost.met.no!
There are  245 lines of data in this dataframe.

No missing values found in the weather data! 



'Fjern # under for å se datasettet'

In [4]:
# ------------------------------------------
# AIR QUALITY DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the air quality data
threshold = 95 # The threshold below which we will not use the data for analysis (uptime % of day)
file_path = '../Data/luftkvalitet_trondheim_dag.csv' #Choose the file to get air quality data from

# Run the module for importing the air quality data, and save it to a dataframe
%run nilu.ipynb
df_quality = get_nilu()

# Run the module for cleaning up and checking the air quality data
'''Per nå sjekker den bare hvilke rader som har for dårlig dekning til å brukes i analyse '''
%run air_data.ipynb
exclude_data=show_zeroes(df_quality)

# Displaying the air quality data as a table
'''Fjern # under for å se datasettet'''
#pretty_data(df_quality)

Data collected from nilu.com!
There are  367 lines of data in this dataframe.

           Tid  Elgeseter NO µg/m³ Day  Dekning  Elgeseter NO2 µg/m³ Day  \
0   2024-02-19               21.319116    100.0                47.910158   
1   2024-02-20               36.586944    100.0                48.447275   
2   2024-02-21               28.709966    100.0                43.229089   
3   2024-02-22                8.784671    100.0                16.795990   
4   2024-02-23                4.420641    100.0                11.082582   
5   2024-02-24                1.929630    100.0                 7.355437   
6   2024-02-25                5.334206    100.0                17.168342   
7   2024-02-26                5.787199    100.0                22.636717   
8   2024-02-27               21.400212     96.0                30.552252   
9   2024-02-28               11.161719    100.0                27.314969   
10  2024-02-29                2.907695    100.0                10.109803   
11  2024-

'Fjern # under for å se datasettet'

In [3]:
# *** EVERYTHING below this line is purely for local testing purposes of this specific file ***
'''Fjern # foran de linjene under som kjører en print funksjon, for å se om det virker :)'''

    # Print the first bit of the weather data to confirm the function is working
#print(df_weather.head())

    # Print the index list of the excluded air quality data:
#print('Rows with poor or no coverage can be found at the following rows: \n',exclude_data)

'Fjern # foran de linjene under som kjører en print funksjon, for å se om det virker :)'